## **문서 벡터 저장소, Vector Stores**

In [1]:
# !pip install langchain langchain-community langchain-openai langchain-chroma tiktoken pypdf sentence_transformers langchain-text-splitters

### **Langchain-Chroma 문서 저장 및 유사 문서 검색**

In [2]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv('../.env')

# 2. 환경 변수 읽기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

openai_embedding=OpenAIEmbeddings(model = 'text-embedding-3-small')

loader = PyPDFLoader(r"../../../data/Langchain-RAG/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

db = Chroma.from_documents(docs, openai_embedding)

In [4]:
query = "대통령의 임기는?"
#유사 문서 검색
docs = db.similarity_search(query)
print(docs[0])

page_content='제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
 
제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로
그 권한을 대행한다.
 
제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수
있다.
 
제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.
 
제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.
②국군의 조직과 편성은 법률로 정한다.
 
제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여
대통령령을 발할 수 있다.
 
제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의' metadata={'creationdate': '2024-04-01T21:26:24+09:00', 'total_pages': 14, 'producer': 'iText 2.1.7 by 1T3XT', 'page_label': '7', 'creator': 'PyPDF', 'page': 6, 'moddate': '2024-04-01T21:26:24+09:00', 'source': '../../../data/Langchain-RAG/대한민국헌법(헌법)(제00010호)(19880225).pdf'}


In [5]:
#유사 문서 검색 및 유사도 출력
db.similarity_search_with_score(query)

[(Document(metadata={'source': '../../../data/Langchain-RAG/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'creationdate': '2024-04-01T21:26:24+09:00', 'page_label': '7', 'moddate': '2024-04-01T21:26:24+09:00', 'page': 6, 'creator': 'PyPDF', 'total_pages': 14, 'producer': 'iText 2.1.7 by 1T3XT'}, page_content='제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.\n \n제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로\n그 권한을 대행한다.\n \n제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수\n있다.\n \n제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.\n \n제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.\n②국군의 조직과 편성은 법률로 정한다.\n \n제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여\n대통령령을 발할 수 있다.\n \n제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의'),
  1.0359079837799072),
 (Document(metadata={'total_pages': 14, 'page': 7, 'creationdate': '2024-04-01T21:26:24+09:00', 'creator': 'PyPDF', 'source': '../../../data/Langchain-RAG/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'moddate': '2024-04

**[벡터DB를 로컬 디스크에 저장하고 로드하기]**

In [6]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

#HuggingfaceEmbedding 함수로 Open source 임베딩 모델 로드
model_name = "jhgan/ko-sroberta-multitask"
ko_embedding= HuggingFaceEmbeddings(
    model_name=model_name
)

#save to disk
db2 = Chroma.from_documents(docs, ko_embedding, persist_directory="./chroma_db")

# load from disk
db3 = Chroma(persist_directory="./chroma_db", embedding_function=ko_embedding)

query = "대통령의 임기는?"
result = db3.similarity_search(query)
print(result[0].page_content)

C:\Users\user\AppData\Local\Temp\ipykernel_20412\3953570494.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  ko_embedding= HuggingFaceEmbeddings(


제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
 
제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로
그 권한을 대행한다.
 
제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수
있다.
 
제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.
 
제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.
②국군의 조직과 편성은 법률로 정한다.
 
제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여
대통령령을 발할 수 있다.
 
제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의


### **Chroma DB API를 활용한 문서 관리**

**Collection 객체 생성과 문서 저장**

In [7]:
import chromadb
#collection을 저장할 경로 지정
client = chromadb.PersistentClient(path="collection_example")
#client가 잘 연결되어 있는지 확인
client.heartbeat()

1761008998857560000

**Collection 생성**

In [8]:
from chromadb.utils import embedding_functions
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
# OpenAI 임베딩
openai_embedding = OpenAIEmbeddings(model='text-embedding-3-small')

# HuggingFace 임베딩
hf_embedding = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="jhgan/ko-sbert-nli")

# 사용할 임베딩 선택
embedding_function = hf_embedding

# 컬렉션 생성
collection = client.create_collection(
    name="korean_law",
    embedding_function=embedding_function
)

**Collection에 문서 임베딩 저장**

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader


# load the document and split it into chunks
loader = PyPDFLoader(r"../../../data/Langchain-RAG/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

# split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

collection.add(
    ids = [str(i) for i in range(len(docs))],
    documents=[i.page_content for i in docs],
    metadatas=[i.metadata for i in docs]
)

**Collection 로드하기**

In [15]:
#name에 collection 이름, embedding_function은 collection 저장 시 지정한 임베딩 모델 지정
collection = client.get_collection(name="korean_law", embedding_function=embedding_function)
collection

Collection(name=korean_law)

**Collection 내 문서 검색**

In [16]:
#1페이지에서 직업 선택의 자유와 유사한 청크 3개 검색
collection.query(
    query_texts=["직업 선택의 자유"],
    n_results=3,
    where={"page": 1},
)

{'ids': [['7', '5', '4']],
 'embeddings': None,
 'documents': [['제15조 모든 국민은 직업선택의 자유를 가진다.\n \n제16조 모든 국민은 주거의 자유를 침해받지 아니한다. 주거에 대한 압수나 수색을 할 때에는 검사의 신청에 의하여 법\n관이 발부한 영장을 제시하여야 한다.\n \n제17조 모든 국민은 사생활의 비밀과 자유를 침해받지 아니한다.\n \n제18조 모든 국민은 통신의 비밀을 침해받지 아니한다.\n \n제19조 모든 국민은 양심의 자유를 가진다.\n \n제20조 ①모든 국민은 종교의 자유를 가진다.\n②국교는 인정되지 아니하며, 종교와 정치는 분리된다.',
   '제12조 ①모든 국민은 신체의 자유를 가진다. 누구든지 법률에 의하지 아니하고는 체포ㆍ구속ㆍ압수ㆍ수색 또는 심문\n을 받지 아니하며, 법률과 적법한 절차에 의하지 아니하고는 처벌ㆍ보안처분 또는 강제노역을 받지 아니한다.\n②모든 국민은 고문을 받지 아니하며, 형사상 자기에게 불리한 진술을 강요당하지 아니한다.\n③체포ㆍ구속ㆍ압수 또는 수색을 할 때에는 적법한 절차에 따라 검사의 신청에 의하여 법관이 발부한 영장을 제시\n하여야 한다. 다만, 현행범인인 경우와 장기 3년 이상의 형에 해당하는 죄를 범하고 도피 또는 증거인멸의 염려가\n있을 때에는 사후에 영장을 청구할 수 있다.\n④누구든지 체포 또는 구속을 당한 때에는 즉시 변호인의 조력을 받을 권리를 가진다. 다만, 형사피고인이 스스로\n변호인을 구할 수 없을 때에는 법률이 정하는 바에 의하여 국가가 변호인을 붙인다.\n⑤누구든지 체포 또는 구속의 이유와 변호인의 조력을 받을 권리가 있음을 고지받지 아니하고는 체포 또는 구속을',
   '법제처                                                            2                                                       국가법령정보센터\n대한민국헌법\n \n 

**조건부 문서 검색**

In [22]:
#5페이지 이후의 청크 중에서 직업 선택의 자유와 관련한 문서 3개 검색
# $eq - 일치 (string, int, float)
# $ne - 불일치 (string, int, float)
# $gt - 초과 (int, float)
# $gte - 이상 (int, float)
# $lt - 미만 (int, float)
# $lte - 이하 (int, float)
collection.query(
    query_texts=["직업 선택의 자유"],
    n_results=3,
    where={"page": {"$gte": 5}}
)


{'ids': [['47', '50', '45']],
 'embeddings': None,
 'documents': [['법제처                                                            13                                                       국가법령정보센터\n대한민국헌법\n⑤국가는 농ㆍ어민과 중소기업의 자조조직을 육성하여야 하며, 그 자율적 활동과 발전을 보장한다.\n \n제124조 국가는 건전한 소비행위를 계도하고 생산품의 품질향상을 촉구하기 위한 소비자보호운동을 법률이 정하는 바\n에 의하여 보장한다.\n \n제125조 국가는 대외무역을 육성하며, 이를 규제ㆍ조정할 수 있다.\n \n제126조 국방상 또는 국민경제상 긴절한 필요로 인하여 법률이 정하는 경우를 제외하고는, 사영기업을 국유 또는 공유\n로 이전하거나 그 경영을 통제 또는 관리할 수 없다.\n \n제127조 ①국가는 과학기술의 혁신과 정보 및 인력의 개발을 통하여 국민경제의 발전에 노력하여야 한다.\n②국가는 국가표준제도를 확립한다.',
   '헌법에 의하여 선임방법이나 임명권자가 변경된 공무원과 대법원장 및 감사원장은 이 헌법에 의하여 후임자가 선\n임될 때까지 그 직무를 행하며, 이 경우 전임자인 공무원의 임기는 후임자가 선임되는 전일까지로 한다.\n②이 헌법시행 당시의 대법원장과 대법원판사가 아닌 법관은 제1항 단서의 규정에 불구하고 이 헌법에 의하여 임명\n된 것으로 본다.\n③이 헌법 중 공무원의 임기 또는 중임제한에 관한 규정은 이 헌법에 의하여 그 공무원이 최초로 선출 또는 임명된\n때로부터 적용한다.',
   '지하며, 경제주체간의 조화를 통한 경제의 민주화를 위하여 경제에 관한 규제와 조정을 할 수 있다.\n \n제120조 ①광물 기타 중요한 지하자원ㆍ수산자원ㆍ수력과 경제상 이용할 수 있는 자연력은 법률이 정하는 바에 의하\n여 일정한 기간 그 채취ㆍ개발 또는 이용을 특허할 수 있다.\

In [18]:
collection.query(
    query_texts=["직업 선택의 자유"],
    n_results=3,
    where={"page": 1},
    where_document={"$contains": "직업"}
)

{'ids': [['7']],
 'embeddings': None,
 'documents': [['제15조 모든 국민은 직업선택의 자유를 가진다.\n \n제16조 모든 국민은 주거의 자유를 침해받지 아니한다. 주거에 대한 압수나 수색을 할 때에는 검사의 신청에 의하여 법\n관이 발부한 영장을 제시하여야 한다.\n \n제17조 모든 국민은 사생활의 비밀과 자유를 침해받지 아니한다.\n \n제18조 모든 국민은 통신의 비밀을 침해받지 아니한다.\n \n제19조 모든 국민은 양심의 자유를 가진다.\n \n제20조 ①모든 국민은 종교의 자유를 가진다.\n②국교는 인정되지 아니하며, 종교와 정치는 분리된다.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'page': 1,
    'creator': 'PyPDF',
    'moddate': '2024-04-01T21:26:24+09:00',
    'creationdate': '2024-04-01T21:26:24+09:00',
    'page_label': '2',
    'total_pages': 14,
    'source': '../../../data/Langchain-RAG/대한민국헌법(헌법)(제00010호)(19880225).pdf',
    'producer': 'iText 2.1.7 by 1T3XT'}]],
 'distances': [[0.5363096594810486]]}